In [1]:
import os
import torch
import cudf
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType
from langchain.memory import ConversationBufferMemory
from langchain.schema import Document
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

# OpenAI API 키를 환경 변수에서 로드
openai_api_key = os.getenv("OPENAI_API_KEY")

# Parquet 파일 경로 설정
file_path = 'merged_data.parquet'

# Parquet 파일을 cudf로 로드
df = cudf.read_parquet(file_path)

# 결측치(NaN)를 'N/A'로 대체
df = df.fillna('N/A')

# 'content' 열을 벡터화하여 생성
df['content'] = (
    'filename: ' + df['filename'].astype(str) + '\n' +
    'date: ' + df['date'].astype(str) + '\n' +
    'conference_number: ' + df['conference_number'].astype(str) + '\n' +
    'question_number: ' + df['question_number'].astype(str) + '\n' +
    'meeting_name: ' + df['meeting_name'].astype(str) + '\n' +
    'generation_number: ' + df['generation_number'].astype(str) + '\n' +
    'committee_name: ' + df['committee_name'].astype(str) + '\n' +
    'meeting_number: ' + df['meeting_number'].astype(str) + '\n' +
    'session_number: ' + df['session_number'].astype(str) + '\n' +
    'agenda: ' + df['agenda'].astype(str) + '\n' +
    'law: ' + df['law'].astype(str) + '\n' +
    'qna_type: ' + df['qna_type'].astype(str) + '\n' +
    'context: ' + df['context'].astype(str) + '\n' +
    'questioner_name: ' + df['questioner_name'].astype(str) + '\n' +
    'questioner_affiliation: ' + df['questioner_affiliation'].astype(str) + '\n' +
    'questioner_position: ' + df['questioner_position'].astype(str) + '\n' +
    'question_comment: ' + df['question_comment'].astype(str) + '\n' +
    'answerer_name: ' + df['answerer_name'].astype(str) + '\n' +
    'answerer_affiliation: ' + df['answerer_affiliation'].astype(str) + '\n' +
    'answerer_position: ' + df['answerer_position'].astype(str) + '\n' +
    'answer_comment: ' + df['answer_comment'].astype(str) + '\n'
)

# cudf DataFrame을 pandas DataFrame으로 변환
# 이 과정은 GPU의 이점을 유지하면서 데이터 변환을 효율적으로 처리할 수 있도록 합니다.
pandas_df = df.to_pandas()

# 'content'와 메타데이터를 분리하여 리스트로 변환
content_list = pandas_df['content'].tolist()
metadata_list = pandas_df.drop(columns=['content']).to_dict('records')

# Document 객체 생성
documents = [Document(page_content=content, metadata=metadata) for content, metadata in zip(content_list, metadata_list)]

# 문서 분할 및 임베딩 생성
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_docs = text_splitter.split_documents(documents)

# CUDA 사용 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 임베딩 모델 설정 (GPU 사용)
embedding_model_name = 'sentence-transformers/all-MiniLM-L6-v2'
embedding_model = HuggingFaceEmbeddings(
    model_name=embedding_model_name,
    model_kwargs={'device': device}
)

# FAISS 인덱스 설정
index_path = 'faiss_index'

# FAISS 인덱스 생성 또는 로드
if os.path.exists(index_path):
    print("저장된 FAISS 인덱스를 로드합니다...")
    vectorstore = FAISS.load_local(
        index_path,
        embeddings=embedding_model,
        allow_dangerous_deserialization=True
    )
else:
    print("FAISS 인덱스를 생성합니다...")
    vectorstore = FAISS.from_documents(split_docs, embedding_model)
    print(f"총 {len(split_docs)}개의 문서가 인덱스에 추가되었습니다.")
    vectorstore.save_local(index_path)


# Retriever 생성
retriever = vectorstore.as_retriever(search_kwargs={"k": 6})

# 도구 함수 정의
def parquet_search_tool(input_text):
    docs = retriever.get_relevant_documents(input_text)
    if docs:
        summaries = [doc.page_content for doc in docs]
        return '\n\n'.join(summaries)
    else:
        return "해당하는 정보를 찾을 수 없습니다."

# 도구 정의
parquet_search_tool = Tool(
    name="ParquetSearch",
    func=parquet_search_tool,
    description=(
        "Parquet 문서에서 특정 인물이나 이슈에 대한 정보를 검색하는 도구입니다. "
        "회의 번호, 회의명, 대수, 위원회명, 안건, 법률, 질의응답 유형, 내용, 요약된 내용, "
        "질문자, 질문 등의 필드를 요약하여 제공합니다."
    )
)

# 도구 목록 생성
tools = [parquet_search_tool]

# LLM 정의
llm = ChatOpenAI(model_name="gpt-4", temperature=0, openai_api_key=openai_api_key)

# 메모리 설정
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# 에이전트 초기화
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
    memory=memory,
)

# 시스템 프롬프트 설정
agent.agent.llm_chain.prompt.messages[0].prompt.template = (
    "반드시 한국어로 답변해 주세요. "
    "당신은 국회 Parquet 문서에서 인물이나 이슈에 대한 정보를 찾아주는 AI 비서입니다. "
    "사용자가 특정 인물이나 이슈에 대해 질문하면, ParquetSearch 도구를 사용하여 해당 인물이 언급된 회의 정보를 찾아야 합니다. "
    "회의 번호, 회의명, 대수, 위원회명, 안건, 법률, 질의응답 유형, 내용, 요약된 내용, 질문자, 질문 등의 필드를 포함하여 요약된 응답을 제공하세요. "
    "Parquet 문서에서 정보를 찾을 수 없는 경우, 정중하게 해당 정보를 찾을 수 없음을 알려주세요. "
    "도구의 사용 여부를 언급하지 말고 필요한 정보만 제공하세요."
)

# 에이전트와 대화하는 함수 정의
def chat_with_agent(user_input):
    response = agent.run(input=user_input)
    return response

# 예시 질문
user_input = "김영삼 대통령이 언급된 회의에 대한 정보를 알려주세요. 반드시 안건 이름과 날짜, 어떤 회의였는지, 몇 대 국회였는지 알려주세요. 질문자와 답변자에 대한 정보와 어떤 안건에 대해 이야기가 오고 갔는지 대화문도 알려주세요."
response = chat_with_agent(user_input)
print("Assistant:", response)


/tmp/ipykernel_210069/3080195416.py:74: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
/home/elicer/.local/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


FAISS 인덱스를 생성합니다...
총 26975개의 문서가 인덱스에 추가되었습니다.


/tmp/ipykernel_210069/3080195416.py:124: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-4", temperature=0, openai_api_key=openai_api_key)
/tmp/ipykernel_210069/3080195416.py:127: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
/tmp/ipykernel_210069/3080195416.py:130: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat



> Entering new AgentExecutor chain...
```json
{
    "action": "ParquetSearch",
    "action_input": "김영삼"
}
```
Observation: answerer_name: 이희범
answerer_affiliation: 
answerer_position: 2018평창동계올림픽대회및동계패럴림픽대회조직위원장
answer_comment: 예, 알고 있습니다.   국민안전처를 중심으로 해서 저희 조직위원회하고 통합안전망체제를 지금 하고 있고요. 몇 차례 회의도 지금 하고 있습니다.

answerer_name: 김현숙
answerer_affiliation: 새만금개발청
answerer_position: 청장
answer_comment: 아니요, 지금 말씀을……

answerer_name: 홍남기
answerer_affiliation: 부총리겸기획재정부
answerer_position: 장관
answer_comment: 위원님 지금 말씀 주신 대로 탄소중립은 저희가 선택할 수 있는 사안이 아니고 필수로 가야 되고요. 이것을……

answerer_name: 홍남기
answerer_affiliation: 기획재정부
answerer_position: 부총리겸기획재정부장관
answer_comment: 예.

questioner_name: 김삼화
questioner_affiliation: 
questioner_position: 소위원장
question_comment: 수고하셨습니다.   다음, 강병구 교수님 준비하신 내용이 있으면 말씀해 주십시오.
answerer_name:  강병구
answerer_affiliation: 
answerer_position: 진술인

questioner_name: 유상범
questioner_affiliation: 
questioner_position: 의원
question_comment: 답변하기 곤란하신가요?
answerer_name: 김부겸
answerer_affil

/tmp/ipykernel_210069/3080195416.py:102: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(input_text)


```json
{
    "action": "Final Answer",
    "action_input": "김영삼 대통령에 대한 회의 정보를 찾았습니다. 이희범이라는 인물이 2018평창동계올림픽대회및동계패럴림픽대회조직위원장으로서 국민안전처와 조직위원회가 통합안전망체제를 구축하고 있다고 언급하였습니다. 또한 김현숙 새만금개발청 청장, 홍남기 부총리겸기획재정부 장관, 홍남기 기획재정부 부총리겸기획재정부장관 등이 참여하였습니다. 김삼화 소위원장이 강병구 교수에게 발언을 요청하였고, 유상범 의원이 김부겸 국무총리에게 질문을 하였습니다. 하지만 회의의 날짜, 안건 이름, 회의명, 몇 대 국회였는지에 대한 정보는 찾을 수 없었습니다."
}
```

> Finished chain.
Assistant: 김영삼 대통령에 대한 회의 정보를 찾았습니다. 이희범이라는 인물이 2018평창동계올림픽대회및동계패럴림픽대회조직위원장으로서 국민안전처와 조직위원회가 통합안전망체제를 구축하고 있다고 언급하였습니다. 또한 김현숙 새만금개발청 청장, 홍남기 부총리겸기획재정부 장관, 홍남기 기획재정부 부총리겸기획재정부장관 등이 참여하였습니다. 김삼화 소위원장이 강병구 교수에게 발언을 요청하였고, 유상범 의원이 김부겸 국무총리에게 질문을 하였습니다. 하지만 회의의 날짜, 안건 이름, 회의명, 몇 대 국회였는지에 대한 정보는 찾을 수 없었습니다.


In [5]:
# Retriever 생성
retriever = vectorstore.as_retriever(search_kwargs={"k": 6})

# Retriever가 반환하는 문서 확인
test_query = "김영삼 대통령"
docs = retriever.get_relevant_documents(test_query)

if docs:
    print(f"검색된 문서 수: {len(docs)}")
    for doc in docs:
        print("Content:\n", doc.page_content)
        print("Metadata:\n", doc.metadata)
        print("-" * 50)
else:
    print("검색된 문서가 없습니다.")


검색된 문서 수: 6
Content:
 당시에 외국으로 출국해서 지금 외국에 체류 중인 것으로 알고 있습니다. 그러니까 국민들 입장에서는 도저히 그 상황을 받아들이기가 어렵지요.   허 전 회장 전 사위가 대표를 맡고 있던 건설사로부터 70억 원 채권을 넘겨받은 사건 지금 경찰에서 수사하고 있습니까? 용인에서 있었던 사건인데 서울중앙지검으로 가 있는 것으로 알고 있습니다. 지금은 서울중앙지검에 갔어요? 광주지검하고는 그러면 지금…… 강제집행면탈로 고소되어서 서울중앙지검에 있는 것으로 알고 있습니다.
Metadata:
 {'filename': 'SRC_20대_2018_2018년10월23일_국정감사_법제사법위원회_0006(048956).xlsx', 'date': '2018년10월23일(화)', 'conference_number': '048956', 'question_number': '0006', 'meeting_name': '국정감사', 'generation_number': '20', 'committee_name': '법제사법위원회', 'meeting_number': '2018', 'session_number': '2018년10월23일', 'agenda': '감사개시', 'law': '', 'qna_type': '추출형', 'context': '수원을 지역 백혜련 위원입니다.  요즘 사법농단 사태로 인해서 검찰청의 영장기각률이 굉장히 올라갔다고 하는데 특히 서울중앙지검을 기준으로 해서는 예전보다 두 배 이상 영장기각률이 상승했다고 합니다.   그런데 혹시 지금 지역의 각 검찰청들은 영장기각률이 예년에 비해서 특별하게 다른 점이 있다든지 그 점에 대해서 각 검사장님들이 지역 상황을 얘기해 주셨으면 좋겠어요. 청주지검장님부터 대전지검장님, 차례대로 지검장님들이. 저희 광주지검은 영장기각률이 동급 청보다 3% 정도 낮은 상황입니다. 수원을 지역 백혜련 위원입니다.  요즘 사법농단 사태로 인해서 검찰청의 영장기각률이 굉장히 올라갔다고 하는데 특히 서울중앙지검을 기준으로 

In [6]:
# Parquet 파일을 cudf로 로드
df = cudf.read_parquet(file_path)

# 결측치(NaN)를 'N/A'로 대체
df = df.fillna('N/A')

# 'content' 열을 벡터화하여 생성
df['content'] = (
    'filename: ' + df['filename'].astype(str) + '\n' +
    'date: ' + df['date'].astype(str) + '\n' +
    'conference_number: ' + df['conference_number'].astype(str) + '\n' +
    'question_number: ' + df['question_number'].astype(str) + '\n' +
    'meeting_name: ' + df['meeting_name'].astype(str) + '\n' +
    'generation_number: ' + df['generation_number'].astype(str) + '\n' +
    'committee_name: ' + df['committee_name'].astype(str) + '\n' +
    'meeting_number: ' + df['meeting_number'].astype(str) + '\n' +
    'session_number: ' + df['session_number'].astype(str) + '\n' +
    'agenda: ' + df['agenda'].astype(str) + '\n' +
    'law: ' + df['law'].astype(str) + '\n' +
    'qna_type: ' + df['qna_type'].astype(str) + '\n' +
    'context: ' + df['context'].astype(str) + '\n' +
    'questioner_name: ' + df['questioner_name'].astype(str) + '\n' +
    'questioner_affiliation: ' + df['questioner_affiliation'].astype(str) + '\n' +
    'questioner_position: ' + df['questioner_position'].astype(str) + '\n' +
    'question_comment: ' + df['question_comment'].astype(str) + '\n' +
    'answerer_name: ' + df['answerer_name'].astype(str) + '\n' +
    'answerer_affiliation: ' + df['answerer_affiliation'].astype(str) + '\n' +
    'answerer_position: ' + df['answerer_position'].astype(str) + '\n' +
    'answer_comment: ' + df['answer_comment'].astype(str) + '\n'
)

# cudf DataFrame을 pandas DataFrame으로 변환
pandas_df = df.to_pandas()

# 데이터 확인
print(pandas_df.head())


                                            filename            date  \
0  SRC_16대_2000_2000년10월20일_국정감사_교육위원회_0001(03004...  2000年10月20日(金)   
1  SRC_16대_2000_2000년10월23일_국정감사_행정자치위원회-제1반_0001...  2000年10月23日(月)   
2  SRC_16대_2000_2000년10월30일_국정감사_법제사법위원회_0001(028...  2000年10月30日(月)   
3  SRC_16대_2000_2000년10월31일_국정감사_통일외교통상위원회-아주반_00...  2000年10月31日(火)   
4  SRC_16대_2000_2000년11월02일_국정감사_재정경제위원회_0001(028...   2000年11月2日(木)   

  conference_number question_number meeting_name generation_number  \
0            030043            0001         국정감사                16   
1            028685            0001         국정감사                16   
2            028928            0001         국정감사                16   
3            029591            0001         국정감사                16   
4            028942            0001         국정감사                16   

  committee_name meeting_number session_number agenda  ...  \
0          교육위원회           2000    2000년10월20일   감사개시  ...   
1    행정자치위원회-제1반      

In [4]:
# Document 객체 생성
documents = [Document(page_content=content, metadata=metadata) for content, metadata in zip(content_list, metadata_list)]

# 일부 Document 객체 출력
for doc in documents[:3]:
    print("Content:\n", doc.page_content)
    print("Metadata:\n", doc.metadata)
    print("-" * 50)


Content:
 filename: SRC_16대_2000_2000년10월20일_국정감사_교육위원회_0001(030043).xlsx
date: 2000年10月20日(金)
conference_number: 030043
question_number: 0001
meeting_name: 국정감사
generation_number: 16
committee_name: 교육위원회
meeting_number: 2000
session_number: 2000년10월20일
agenda: 감사개시
law: 
qna_type: 추출형
context: 任鍾晳 위원입니다.  자료준비와 국정감사를 직접 받으시느라고 많이 피로하실 줄 압니다. 심심한 위로와 감사의 인사를 함께 드립니다.   저는 시간내에 일문일답으로 하겠습니다. 혹시 부족한 것이 있으면 서면으로 하겠습니다.   먼저 인천교육청에 질의 드리겠습니다.   우리가 씨랜드에서도 볼 수 있듯이 학생안전권은 학습권이나 복지권보다 훨씬 중요하다고 생각합니다. 앞서 金貞淑 위원님께서 말씀하셨지만 학생들의 유치원, 초ㆍ중ㆍ고등학교의 통학버스가 종합보험 특별약관에 미가입된 차량들이 전국에 18.2%나 됩니다.   인천도 27.3%가 종합보험 특별약관에 미가입되어서 불의의 사고시 굉장한 피해가 염려되고 있습니다. 서울과 대구, 광주, 경기도는 미가입문제가 깨끗이 정리되어 있고 나머지 지역은 좀 많은 데도 있고 적은 데도 있습니다. 인천은 전국 평균보다 약 10% 높은데 이것을 즉시 해결하는데 어떤 어려움이 있습니까? 말씀드리겠습니다.   인천시내에 초등학교 버스 미가입은 공립의 경우는 없고 사립은 거의 해결됐고 대부분이 유치원입니다. 사립유치원의 버스는 미가입 상태가 많습니다. 고등학교도 꽤 있는데요? 현재 보험에 미가입된 상태는 없습니다. 이것은 제가 자료를 요청해서 받은 것인데 종합보험 특별약관에 인천지역은 중학교만 깨끗할 뿐 초등학교나 고등학교는 미가입 차량들이 있습니다.   공ㆍ사립 불문하고 이것을 즉시 가입시키는 행정조치를 하는데 어려